# Домашнее задание № 4. Языковые модели

In [31]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
from collections import Counter
import pandas as pd

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [2]:
dvach = open('data/2ch_corpus.txt').read()
news = open('data/lenta.txt').read()

In [284]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [285]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [294]:
#вот тут, наверное, добавить еще один тег <start>? 
#start0 чтобы после него точно шел только start, а после start только слово (а не новый тег start)
sentences_dvach = [['<start0>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:5000000])]
sentences_news = [['<start0>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [295]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))


unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [15]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [296]:
# матрица слова на слова (инициализируем нулями)
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                         len(unigrams_dvach)))
#индексы для униграм и биграм
id2bigr_dvach = list(bigrams_dvach)
bigr2id_dvach = {bigram:i for i, bigram in enumerate(id2bigr_dvach)}
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}


# заполняем матрицу
for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = ' '.join([word1, word2])
    # на пересечение биграмы и слова ставим вероятность встретить второе после первого
    matrix_dvach[bigr2id_dvach[bigram], word2id_dvach[word3]] =  (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[bigram])
    

In [159]:
#pd.DataFrame.sparse.from_spmatrix(matrix_dvach)

In [162]:
# то же самое для другого корпуса
# матрица слова на слова (инициализируем нулями)
matrix_news = lil_matrix((len(bigrams_news), 
                         len(unigrams_news)))

#индексы для униграм и биграм
id2bigr_news = list(bigrams_news)
bigr2id_news = {bigram:i for i, bigram in enumerate(id2bigr_news)}
id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}


for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = ' '.join([word1, word2])
    # на пересечении биграмы и слова ставим вероятность встретить второе после первого
    matrix_news[bigr2id_news[bigram], word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

In [156]:

def generate(matrix, id2word, id2bigr, bigr2id, n=100, start='<start0> <start>'):
    text = []
    current_idx = bigr2id[start]
    
    for i in range(n):
#         p = matrix[current_idx].toarray()[0]
#         p = np.asarray(p).astype('float64')
#         p = p / np.sum(p)
#         chosen = np.random.choice(matrix.shape[1], p=p)
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen]) 
        #из последнего слова предыдущей биграмы и добавленного слова делаем новую биграму, получаем новый current_idx
        chosen_bg = bigr2id[' '.join([id2bigr[current_idx].split()[1], id2word[chosen]])]
        if id2word[chosen] == '<end>':
            chosen_bg = bigr2id['<start0> <start>']
        
        current_idx = chosen_bg
    
    return ' '.join(text)

In [157]:
print(generate(matrix_dvach, id2word_dvach, id2bigr_dvach, bigr2id_dvach).replace('<end>', '\n'))

оптимизон нормальный это такая штука когда тебе в общем-то нихуя 
 с 2009 в завязке 
 ночью вообще все тян не хорошие лол 
 а сколько время то 
 опиши стек для стандартных gui приложений winforms в большинстве городов но это малое от чего умер 
 допустим у меня по 30 лет однозначно уверен что дело в том что сами за 5 к выкинул просто на основе шариата принимает 
 всмысле ты употребляешь говно 
 а есть пидоры 
 давайте я у него в открытом космосе летают мириады мелких объектов по совершенно непредсказуемым орбитам 
 ты в толчок стучался постоянно


## для трех слов

In [163]:
sentences_dvach = [['<start0>'] + ['<start1>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:5000000])]

In [165]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()
fourgrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))
    fourgrams_dvach.update(ngrammer(sentence, n=4))

In [166]:
# матрица слова на слова (инициализируем нулями)
matrix_dvach = lil_matrix((len(trigrams_dvach), 
                         len(unigrams_dvach)))
#индексы для униграм и биграм
id2trigr_dvach = list(trigrams_dvach)
trigr2id_dvach = {tg:i for i, tg in enumerate(id2trigr_dvach)}
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}


# заполняем матрицу
for ngram in fourgrams_dvach:
    word1, word2, word3, word4 = ngram.split()
    trigram = ' '.join([word1, word2, word3])
    # на пересечении триграмы и слова ставим вероятность встретить второе после первого
    matrix_dvach[trigr2id_dvach[trigram], word2id_dvach[word4]] =  (fourgrams_dvach[ngram]/
                                                                     trigrams_dvach[trigram])
    

In [176]:
def generate(matrix, id2word, id2trigr, trigr2id, n=100, start='<start0> <start1> <start>'):
    text = []
    current_idx = trigr2id[start]
    
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen]) 
        #из последних двух слов предыдущей 3грамы и добавленного слова делаем новую триграму, получаем новый current_idx
        chosen_bg = trigr2id[' '.join([id2trigr[current_idx].split()[1], id2trigr[current_idx].split()[2], id2word[chosen]])]
        if id2word[chosen] == '<end>':
            chosen_bg = trigr2id[start]
        current_idx = chosen_bg
    
    return ' '.join(text)

In [179]:
print(generate(matrix_dvach, id2word_dvach, id2trigr_dvach, trigr2id_dvach).replace('<end>', '\n'))

я там не был ни разу 
 как вообще психонавт под лсд выглядит со стороны 
 почему тебе нечего ответить гуманитарий 
 сейчас хочу окончательно спрыгнуть хуй знает получится ли 
 деды за хлеб воевали а она хлеб в пизду суёт и не уважает его ну вообще сувать в пизду член пиздец какой порок штоле мне кажется тебе максимум 16 
 только не это лучше уж жоповирусный чем ротавирусный 
 я как то загорелся его посмотреть залез на свой любимый рутрекер а там нихуя 
 и если честно то я бы его только обоссал 
 ведь строить заново ничего не


In [178]:
print(generate(matrix_dvach, id2word_dvach, id2trigr_dvach, trigr2id_dvach).replace('<end>', '\n'))

ты то злой то печальный то ты в предвкушении чуда то чуть не плачешь от своей безнадежности 
 тебя спросили про теракты в кса а ты ответил про теракты в турции 
 я про то что это мировая тенденция продавленная мировой плутократией еще не означает что он тупой скорее наоборот умный и богатый 
 но вот вам парочка советов 
 • у человека 23 пары хромосом 
 не твоя вот и бесишься 
 пока не решили 
 — isbn 966-00-0604-7 
 даже если есть такой вариант через нейтральные системы кругом из 2-3 систем вполне вариант обойти 
 меня вообще удивляет


In [182]:
sents = generate(matrix_dvach, id2word_dvach, id2trigr_dvach, trigr2id_dvach).split('<end>')
sents

['я сперва пожарил рис минуту в масле на сковороде так что зёрнышки разделились и чутка пожелтели потом добавил три объёма воды и на маленькой скорости варил десять минут ',
 ' просит чтоб опять выходили а то может ты мне так же пытаешься это объяснить и мы оба друг друг не понимаем ',
 ' бывает у некоторых вообще ульт убивает все что увидит и они это пытаются выдать за киберспортивную дисциплину вывод лучше бы они начали разработку 4 варика или нового старкрафта или хотя бы первый шаг сделать начать работать и зарабатывать ',
 ' я был бы не против поменять кое-какие безделицы на патроны']

In [ ]:
#довольно связные тексты)

## перплексия

In [313]:
#по двум предыдущим словам
def compute_join_proba_markov_assumption(phrase, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(phrase)
    for ngram in ngrammer(['<start0>'] + ['<start>'] + tokens + ['<end>'], n=3):
        word1, word2, word3 = ngram.split()
        bigram = ' '.join([word1, word2])
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

In [301]:
#униграмная для сравнения
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)

In [303]:
norm_dvach = normalize(dvach)
vocab_dvach = Counter(norm_dvach)
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})

In [318]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [319]:
#триграмная
ps = []
for sent in sent_tokenize(dvach[-3000:]):
    prob, N = compute_join_proba_markov_assumption(sent, bigrams_dvach, trigrams_dvach)
    if not N:
        continue
    ps.append(perplexity(prob, N))

np.mean(ps)

62601265.468576446

In [ ]:
#очень высокая перплексия....

In [315]:
#униграмная
ps = []
for sent in sent_tokenize(dvach[-3000:]):
    prob, N = compute_joint_proba(sent, probas_dvach)
    if not N:
        continue
    ps.append(perplexity(prob, N))
np.mean(ps)

12046.868014234864

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [321]:
sentences_news = [['<start0>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [323]:
# матрица слова на слова (инициализируем нулями)
matrix_news = lil_matrix((len(bigrams_news), 
                         len(unigrams_news)))

#индексы для униграм и биграм
id2bigr_news = list(bigrams_news)
bigr2id_news = {bigram:i for i, bigram in enumerate(id2bigr_news)}
id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}


for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = ' '.join([word1, word2])
    # на пересечении биграмы и слова ставим вероятность встретить второе после первого
    matrix_news[bigr2id_news[bigram], word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

In [324]:
# сделаем класс чтобы хранить каждый из лучей
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score 

In [325]:
def generate_with_beam_search(matrix, bigr2id, id2word, word2id, n=100, max_beams=5, start='<start0> <start>'):
    # изначально у нас один луч с заданным началом (start по дефолту)
    initial_node = Beam(sequence=start.split(), score=np.log1p(0))
    beams = [initial_node]
    
    for i in range(n):
        # делаем n шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue
            
            # достаем предыдущую биграму из beam.sequence
            last_id = bigr2id[' '.join(beam.sequence[-2:])]
            
            # посмотрим вероятности продолжений для предыдущего слова
            probas = matrix[last_id].toarray()[0]
            
            # возьмем топ самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            for top_id in top_idxs:
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    break
                
                # создадим новый луч на основе текущего и варианта продолжения
                new_sequence = beam.sequence + [id2word[top_id]]
                # скор каждого луча это произведение вероятностей (или сумма логарифмов)
                new_score = beam.score + np.log1p(probas[top_id])
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        # отсортируем лучи по скору и возьмем только топ max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
    
    # в конце возвращаем самый вероятный луч
    best_sequence = max(beams, key=lambda x: x.score).sequence

    
    return ' '.join(best_sequence)

In [326]:
print(generate_with_beam_search(matrix_news, bigr2id_news, id2word_news, word2id_news))

<start0> <start> как сообщает риа новости со ссылкой на источники в правоохранительных органах города интерфаксу сообщили в штабе объединенной группировки войск на северном кавказе генерал-полковник виктор казанцев может получить лишь иностранный гражданин который передал этим летом разразилась похожая эпидемия по данным риа новости со ссылкой на источники в правоохранительных органах города интерфаксу сообщили в штабе объединенной группировки войск на северном кавказе генерал-полковник виктор казанцев может получить лишь иностранный гражданин который передал этим летом разразилась похожая эпидемия по данным риа новости со ссылкой на источники в правоохранительных органах города интерфаксу сообщили в штабе объединенной группировки войск на северном кавказе генерал-полковник виктор казанцев <end>


In [283]:
print(generate_with_beam_search(matrix_news, bigr2id_news, id2word_news, word2id_news, start = 'куда пошли'))

куда пошли деньги мвф для выкупа государственных долгов руководствуясь особенностями ситуации данной страны fr во франции насчитывается более 33 миллионов зараженных вирусом спида в лидеры уверенно вышли республики бывшего ссср лондонскому клубу банков-кредиторов завершился существенным прогрессом но одновременно и противникам ку-клукс-клана было отказано во въезде в старинный русский город появились металлические цилиндры напоминающие поставленные на попа автоцистерны сообщает фан около 2 часов по московскому утра по местному телеканалу ичкерия с заявлением и патриарх московский и всея руси алексия ii не состоится поскольку жители дагестана заявляли что игорь иванов передает риа новости со ссылкой на источники в правоохранительных органах города интерфаксу сообщили в
